# Import libraries

In [1]:
import numpy as np
import pandas as pd
import os

from tqdm import tqdm
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load all images into DataFrame

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iamsouravbanerjee/animal-image-dataset-90-different-animals")

print("Path to dataset files:", path)

100%|██████████| 656M/656M [00:03<00:00, 187MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/iamsouravbanerjee/animal-image-dataset-90-different-animals/versions/5


In [3]:
IMAGE_EXTS = (".jpg", ".jpeg", ".png", ".bmp")

# Function for pushing images with labels into DataFrame
def build_image_df(root_dir):
    rows = []
    for label in os.listdir(root_dir):
        class_dir = os.path.join(root_dir, label)
        if not os.path.isdir(class_dir):
            continue

        for fname in os.listdir(class_dir):
            if fname.lower().endswith(IMAGE_EXTS):
                rows.append({
                    "filepath": os.path.join(class_dir, fname),
                    "label": label,
                    "filename": fname
                })
    return pd.DataFrame(rows)

df = build_image_df(path + "/animals/animals")
df["label"].value_counts()

,count
label,
hummingbird,60
cat,60
pelecaniformes,60
parrot,60
beetle,60
...,...
wombat,60
flamingo,60
seal,60


In [4]:
le = LabelEncoder()
df["label_idx"] = le.fit_transform(df["label"])

num_classes = df["label_idx"].nunique()

# Split dataset into train, val, test dataset
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
# Set up for using gpu to training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
class ImageDFDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["filepath"]).convert("RGB")

        if self.transform:
            img = self.transform(img)

        label = row["label_idx"]
        return img, label

In [7]:
# Rebuild model with backbone resnet18
class AnimalClassification(nn.Module):
    def __init__(self, size_inner=256, droprate=0.2, num_classes=90):
        super(AnimalClassification, self).__init__()

        # Load pretrained ResNet18
        self.base_model = models.resnet18(weights="IMAGENET1K_V1")

        # Freeze backbone
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Remove original FC layer
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()

        # Custom head
        self.inner = nn.Linear(in_features, size_inner)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(droprate)
        self.output_layer = nn.Linear(size_inner, num_classes)

    def forward(self, x):
        x = self.base_model(x)          # (B, 512)
        x = self.inner(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.output_layer(x)
        return x

In [8]:
# Create make model function, later use to tune params
def make_model(learning_rate=0.0001, size_inner=256, droprate=0.3):
    model = AnimalClassification(
        size_inner=size_inner,
        droprate=droprate,
        num_classes=90
    ).to(device)

    optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=learning_rate
    )
    return model, optimizer

In [9]:
# Pre-processing images input
IMG_SIZE = 224
BATCH_SIZE = 20

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=27),  # 0.15 * 180 ≈ 27 độ
    transforms.RandomResizedCrop(
        size=224,
        scale=(0.85, 1.0)  # zoom ~15%
    ),
    transforms.ColorJitter(
        contrast=0.15
    ),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    )
])

In [10]:
# Create DataLoader
full_train_ds = ImageDFDataset(df_full_train, transform=train_transform)
test_ds = ImageDFDataset(df_test, transform=val_transform)

full_train_loader = DataLoader(
    full_train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [11]:
# Create train function
def train_one_epoch(model, loader):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    return running_loss / len(loader), correct / total

In [12]:
# Create validate function
def validate(model, loader):
    model.eval()
    total_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

    return total_loss / len(loader), correct / total


In [13]:
best_val_accuracy = 0.0
checkpoint_path = 'animals_classification_{epoch:02d}_{val_accuracy:.3f}.pth'

EPOCHS = 60
criterion = nn.CrossEntropyLoss()
model, optimizer = make_model(learning_rate=0.0005, size_inner=256, droprate=0.2)

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, full_train_loader)
    val_loss, val_acc = validate(model, test_loader)

    print(
            f"Epoch {epoch+1}/{EPOCHS} | "
            f"Train Acc: {train_acc:.4f} | "
            f"Val Acc: {val_acc:.4f}"
        )

    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        # Format the checkpoint filename with epoch and accuracy
        current_checkpoint_path = checkpoint_path.format(epoch=epoch+1, val_accuracy=val_acc)
        torch.save(model.state_dict(), current_checkpoint_path)
        print(f'Checkpoint saved to {current_checkpoint_path}')

print('Finished Training')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 99.9MB/s]
100%|██████████| 216/216 [01:10<00:00,  3.07it/s]


Epoch 1/60 | Train Acc: 0.2519 | Val Acc: 0.6519
Checkpoint saved to animals_classification_01_0.652.pth


100%|██████████| 216/216 [01:07<00:00,  3.19it/s]


Epoch 2/60 | Train Acc: 0.6049 | Val Acc: 0.7741
Checkpoint saved to animals_classification_02_0.774.pth


...

Epoch 58/60 | Train Acc: 0.9521 | Val Acc: 0.8648


100%|██████████| 216/216 [01:06<00:00,  3.23it/s]


Epoch 59/60 | Train Acc: 0.9498 | Val Acc: 0.8639


100%|██████████| 216/216 [01:06<00:00,  3.25it/s]


Epoch 60/60 | Train Acc: 0.9535 | Val Acc: 0.8880
Checkpoint saved to animals_classification_60_0.888.pth
Finished Training
